In [ ]:
import pandas as pd
import numpy as np
import gensim
import random
import sys
import glob
import os
import datetime
from nltk import sent_tokenize
from nltk import word_tokenize
from scipy.spatial.distance import cosine
import warnings
from gensim.parsing.preprocessing import strip_non_alphanum, stem_text, preprocess_string
from gensim.utils import simple_preprocess
from gensim.models.callbacks import CallbackAny2Vec
from itertools import product
from sklearn.metrics import precision_recall_curve

warnings.simplefilter('ignore')

sys.path.append("../../oats")
from oats.annotation.ontology import Ontology
from oats.distances import pairwise as pw
from oats.utils.utils import flatten

In [ ]:
# Add in the section here for creating the small validation set.
# This creates a dataframe that has all the parent child term pairs in it including synonyms, and excluding
# the term pairs that have an explicit overlap in a token because that's not useful for embedding validation.

# Load the ontology and term information.
path = "../ontologies/pato.obo"
ont = Ontology(path)
term_ids_and_names = [(t.id,t.name) for t in ont.terms() if "obsolete" not in t.name]

# Also including all the synonym information here.
term_ids_and_names_with_synonyms = []
for i,name in term_ids_and_names:
    term_ids_and_names_with_synonyms.append((i," ".join(ont.term_to_tokens[i])))

key_to_annotations = {i:[x[0]] for i,x in enumerate(term_ids_and_names)}
key_to_term_id = {i:x[0] for i,x in enumerate(term_ids_and_names)}
key_to_text_string = {i:x[1] for i,x in enumerate(term_ids_and_names_with_synonyms)}
key_to_preprocessed_text_string = {i:" ".join(preprocess_string(s)) for i,s in key_to_text_string.items()}

# Get mappings that define which terms are very close to which others ones in the ontology structure.
parents = {}
children = {}
for term in ont.terms():
    parents[term.id] = [t.id for t in term.superclasses(with_self=False, distance=1)]
    children[term.id] = [t.id for t in term.subclasses(with_self=False, distance=1)]
siblings = {}
for term in ont.terms():
    siblings[term.id] = flatten([[t for t in children[parent_id] if t!=term.id] for parent_id in parents[term.id]])
assert len(parents) == len(children)
assert len(parents) == len(siblings)
any_close = {}
for key in parents.keys():
    any_close[key] = flatten([parents[key],children[key],siblings[key]])
    any_close[key] = flatten([parents[key],children[key]])

df = pw.with_annotations(key_to_annotations, ont, "jaccard", tfidf=False).edgelist
df = df[df["from"]!=df["to"]]
df["from_id"] = df["from"].map(lambda x: key_to_term_id[x])
df["to_id"] = df["to"].map(lambda x: key_to_term_id[x])
df["from_text"] = df["from"].map(lambda x: key_to_text_string[x])
df["to_text"] = df["to"].map(lambda x: key_to_text_string[x])
df["close"] = df.apply(lambda x: x["to_id"] in any_close[x["from_id"]], axis=1)
df["token_overlap"] = df.apply(lambda x: len(set(x["from_text"].split()).intersection(set(x["to_text"].split())))>0, axis=1)
df = df[(df["token_overlap"]==False) & (df["close"]==True)]
df.head(20)

In [ ]:
word_threshold = 2
df["candidates"] = df.apply(lambda x: (len(x["from_text"].split())>=word_threshold) and (len(x["to_text"].split())>=word_threshold), axis=1)
df = df[df["candidates"]]
df.to_csv("../data/corpus_related_files/closely_related/concepts_candidates.csv", index=False)
df.head(10)
df.shape